<a href="https://colab.research.google.com/github/deangarcia/NLP/blob/main/NLP_Assignment1_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment, you will familiarize yourself with:



*   Python
*   Jupyter Notebooks
*   Google Colab

to develop

* A n-gram based language model, with smoothing

to be able to

* Produce naturalish text
* Rate the perplexity of a text given your model

First we will load in some data.

Provided is code that will download a file and rename it corpus.zip

In [1]:
!wget -O corpus.zip https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1&isAllowed=y 

--2022-02-24 20:14:45--  https://digitalrepository.wheatoncollege.edu/bitstream/handle/11040/24451/Shakespeare%20Corpus%20by%20Play.zip?sequence=1
Resolving digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)... 52.45.246.5
Connecting to digitalrepository.wheatoncollege.edu (digitalrepository.wheatoncollege.edu)|52.45.246.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8536913 (8.1M) [application/octet-stream]
Saving to: ‘corpus.zip’

corpus.zip          100%[===================>]   8.14M  37.7MB/s    in 0.2s    

2022-02-24 20:14:46 (37.7 MB/s) - ‘corpus.zip’ saved [8536913/8536913]



By typing "!" in a notebook, you can use command line prompts

In [2]:
!ls 

corpus.zip  sample_data



Now, we will unzip the corpus


In [3]:
!unzip -o corpus.zip

Archive:  corpus.zip
  inflating: Shakespeare Corpus by Play/henryVIII/henryVIII_SCRBnoCN_noSD.txt  
  inflating: Shakespeare Corpus by Play/henryVIII/henryVIII_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/macbeth_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/juliusCaesar/_DS_Store  
  inflating: Shakespeare Corpus by Play/macbeth/macbeth_SCRBnoCNnoSD_S.txt  
  inflating: Shakespeare Corpus by Play/henryVIII/_DS_Store  
  inflating: Shakespeare Corpus by Play/juliusCaesar/juliusCaesar_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/edwardIII/_DS_Store  
  inflating: Shakespeare Corpus by Play/henryV/_DS_Store  
  inflating: Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_noSCRB.txt  
  inflating: Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_SCRBnoCNnoSD.txt  
  inflating: Shakespeare Corpus by Play/allsWellThatEndsWell/_DS_Stor

We can even have python and the command line interact.

In [4]:
files = !ls Shake*/*/*_noSCRB.txt 


`files` is now a list containing all of the files we wish to use in our data

In [5]:
print(files)

["'Shakespeare Corpus by Play/allsWellThatEndsWell/allsWellThatEndsWell_noSCRB.txt'", "'Shakespeare Corpus by Play/antonyAndCleopatra/antonyAndCleopatra_noSCRB.txt'", "'Shakespeare Corpus by Play/asYouLikeIt/asYouLikeIt_noSCRB.txt'", "'Shakespeare Corpus by Play/comedyOfErrors/comedyOfErrors_noSCRB.txt'", "'Shakespeare Corpus by Play/coriolanus/coriolanus_noSCRB.txt'", "'Shakespeare Corpus by Play/cymbeline/cymbeline_noSCRB.txt'", "'Shakespeare Corpus by Play/edwardIII/edwardIII_noSCRB.txt'", "'Shakespeare Corpus by Play/hamlet/hamlet_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVparti/henryIVparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryIVpartii/henryIVpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryV/henryV_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIII/henryVIII_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIparti/henryVIparti_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartii/henryVIpartii_noSCRB.txt'", "'Shakespeare Corpus by Play/henryVIpartiii/h

Note, there are extra quotes on the files, so we will use list comprehension to remove those

In [6]:
files = [file[1:-1] for file in files]

#STEP 1


* Go through all of the files
  * Load the file `file = open(filename,'r')` 
  * Use the spacy tokenizer to tokenize the text `tokens = tokenizer(file.read())`
  * Loop through the tokens and make everything lowercase `.lower()`
  * Use all of this to make a list (for each file) of lists of strings (the tokens in each file) called `texts` 



  e.g.

  If we read in one file that looked like:
  
    HAMLET: Alas poor Yorick.

    GRAVEDIGGER: Put that skull down.



  The end results should look like:

  `texts = [['<s>', 'HAMLET:', 'Alas', 'poor', 'Yorick.','\n','GRAVEDIGGER:', Put','that', 'skull', 'down.', '</s>']]`
 

In [7]:
!pip install spacy

In [8]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

# Construction 2
from spacy.lang.en import English
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer
texts = []

for filename in files:
  test = []
  filex = open(filename,'r')
  tokens = tokenizer(filex.read())
  for token in tokens:
    test.append(token.lemma_.lower())
  texts.append(test)

#STEP 2

With our files read in, it's time to do some counting!

* Implement the function `get_counts(texts,n)` which takes in a list of list of strings -- `texts` -- and the arity of the n-gram -- `n` 
* `get_counts` should return a dictionary where the keys are the n-grams (as tuples of strings) and the values are the integer counts of all of the found n-grams


Here's a quick primer on tuples in Python

    triple = (1,2,3)
    double = (1,2)
    single = (1,) #note the trailing comma to distinguish from parenethesis 
    null_tuple = tuple() 

    list_triple = [1,2,3]
    tuple_triple = tuple(list_triple)
    


In [9]:
def get_counts(texts,n):
  ngrams = {}
  #Text is a list of lists
  for text in texts:
    # iterate through each list (-n + 1 because of the nested for loop we dont want to go out of range and len(array) doesnt take into account 0)
    for i in range(len(text) - n + 1):
      temp = [] 
      # now iterate through the list j values which is the length of n to save our string values into a tuple
      for j in range(n):
        temp.append(text[i+j])
      tuple_temp = tuple(temp)
      # save the tuple as a key at increment it by one if it exists already 
      if tuple_temp in ngrams.keys():
        ngrams[tuple_temp] += 1
      # or set it to one on the first occurence
      else:
        ngrams[tuple_temp] = 1

  return ngrams

unigrams = get_counts(texts,1)
bigrams = get_counts(texts,2)
trigrams = get_counts(texts,3)


1


#STEP 3

With our counts, it's time to make our first language model.

* Implement the class NGramLM 
  * Initialize the LM
    * First use `get_counts` to get the raw counts of the n-grams up to order `n`
    * Convert these raw counts into probabilities
      * ### Question 1:  What is the relationship between raw counts of n-grams of order n and n-1 and the probabilities of the next word for order n? 
  * Implement the `generate_text` function
    * You are given an optional prompt (a list of strings) -- first convert these to lower case (as the original text was)
    * You should return a list of strings based on randomly choosing according to the probabilities of the language model (using the `random` library)
  * Implement `score_text` which when given a text (a list of strings) returns the perplexity of that text
    * Make sure you don't have any divide by 0 (or `log(0)`) issues -- return `float('inf')` if there is an n-gram not found in the model (note: the perplexity is infinite, not the probability)

In [28]:
import random
import math
random.seed(0)

class NGramLM():
  def __init__(self,texts,n):
    self.xgram_probs = {}
    self.n = n
    self.ngram = {}
    
    # when n = 1 we just need a dictionary of keys that map to probabilities
    if n == 1:
      self.ngram = get_counts(texts,n)
      for key in self.ngram:
        # the probability is the count of the key value divide by the total length of the set
        self.xgram_probs[key] = self.ngram[key] / len(self.ngram)
    else:
      #if not equal to one then we can set our probability according to the context
      self.ngram = get_counts(texts,n)
      nmI_gram = get_counts(texts,n-1)
      
      # if we iterate through all the keys in our ngram we will be able to save our next word along with the probability
      for key in self.ngram:
        temp_dict_list = []
        temp_dict = {}
        # Here we are setting ourselves up to create a dictionary of lists of dictionaries. At the top level 
        # our context is the key, then that key will point to a list of dictionaries whose keys are the next word in the ngram
        # and the value is the probability of that word we do that by taking the count
        temp_dict[key[-1:]] = self.ngram[key] / nmI_gram[key[:n-1]]
        temp_dict_list.append(temp_dict)
        # if the key already exists append to the end of the list
        if key[:n-1] in self.xgram_probs:
          self.xgram_probs[key[:n-1]].append(temp_dict) 
        #if not set up a new dictionary list with the key being the context
        else:
          self.xgram_probs[key[:n-1]] = temp_dict_list

  # we want a dictionary of n-1 grams and then their associated word with the probablity of it occuring        
  def generate_text(self, length, prompt=[]):
    temp = []

    for word in prompt:
      temp.append(word.lower())

    for j in range(length):
      r = random.random()
      accumulator = .0
      break_flag = 0

      if self.n == 1:
        # to generate context here when n == 1 without context I am just randomly grabbing a key value and 
        # adding the probability to see if it falls in the range of the random number if it does then that is the randomly generated word
        while not break_flag:
          key = random.choice(list(self.xgram_probs))
          accumulator += self.xgram_probs[key]
          if accumulator > r:
            for word in list(key):
              temp.append(word)
              break_flag = 1
          if break_flag == 1:
            break
            
      else:
        # otherwise if we have some kick off context for the prompt we can start generating words based off the prompt
        if len(temp) >= (self.n - 1):
          temp_tup = tuple()
          temp_arr = []
          for i in range(self.n - 1):
            temp_arr.append(temp[len(temp) - (self.n - 1 + i)])
          temp_tup = tuple(temp_arr)
          # no we create a tuple that will be used as a key to search our ngram context dictionary
          # if it exists in it we can find a range of probabilities to generate the next word
          if temp_tup in self.xgram_probs:
            # now that we found a matching context key we can iterate through our list for that context to find a 
            # random word
            for i in range(len(self.xgram_probs[temp_tup])):
              #iterate through the keys of the list of dictionaries to get the probablities 
              for tok in self.xgram_probs[temp_tup][i]:
                # add the probability to an accumulator and once that falls into the range of our random variable 
                # we can add that random word to our list
                # for step 5 of interpolation we need to sum up the probabilities times the lamda value to get value to 
                # add to our accumulator 
                accumulator += self.xgram_probs[temp_tup][i][tok]
                if accumulator > r:
                  for word in list(tok):
                    temp.append(word)
                    break_flag = 1
                if break_flag == 1:
                  break
              if break_flag == 1:
                break
          else:
            # if it doesnt exist then we need to generate a random word here
            # in step four instead of generating a random word we put the value <UNK>
            # and in step five for the backoff we search the n - 1 gram for matching context
            key = random.choice(list(self.xgram_probs))
            for i in range(len(self.xgram_probs[key])):
              for tok in self.xgram_probs[key][i]:
                accumulator += self.xgram_probs[key][i][tok]
                if accumulator > r:
                  for word in list(tok):
                    temp.append(word)
                    break_flag = 1
                if break_flag == 1:
                  break
        else:
          # if there is no kick off context then randomly generate a word similar to how when n == 1 only difference is that 
          # we must enter the nested list of dictionaries that pairs up with the context
          key = random.choice(list(self.xgram_probs))
          for i in range(len(self.xgram_probs[key])):
            for tok in self.xgram_probs[key][i]:
              accumulator += self.xgram_probs[key][i][tok]
              if accumulator > r:
                for word in list(tok):
                  temp.append(word)
                  break_flag = 1
              if break_flag == 1:
                break
        
    return temp

  def score_text(self,text):
    perplexity = 0
    prob_sum = 0
    N = len(text)

    if self.n == 1:
      for word in text:
        temp_arr = []
        temp_arr.append(word.lower())
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs:
          prob_sum += math.log(self.xgram_probs[temp_tup], 2)
        else:
          return float('inf')
    else:
      for word in range(len(text) - self.n + 1):
        temp_sum = 0
        temp_arr = []
        for j in range(self.n - 1):
          temp_arr.append(text[word+j].lower())
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs:
          for i in range(len(self.xgram_probs[temp_tup])):
            temp_arr_1 = []
            temp_arr_1.append(text[word + self.n - 1])
            temp_tup_1 = tuple(temp_arr_1)
            if temp_tup_1 in self.xgram_probs[temp_tup][i]:
              temp_sum = self.xgram_probs[temp_tup][i][temp_tup_1] 
        else:
          return float('inf')
        if temp_sum != 0:
          prob_sum += math.log(temp_sum, 2)
    
    #float inf is infinity
    if prob_sum == 0 or N == 0:
      perplexity = float('inf')
    else:
      perplexity = 2 ** (-prob_sum / N)

    return perplexity


In [29]:
for n in [1,2]:
  print(f'Order {n} LM:')
  lm = NGramLM(texts,n)
  
  print(' '.join(lm.generate_text(100)))
  print(' '.join(lm.generate_text(100,['duke', 'of', 'france'])))

  print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
  print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
  print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

Order 1 LM:
. will thee 
   hope we and are that . but , to brutus . achilles when watchings -pron- gainst like and - gathered hath say beauty in her will he prithee 
 dearly gallows enter thou so faulconbridge be will band doth placed ' set propose welshmen can ' dead own shield thy sister spongy hath am laying will in your of usurps speak of is she time how court , crevice she how put fathers ears and with carrion vile to wrack ; for be the best i can offence 
   : not what thyself it they earth
duke of france up offer manners married stay sirrah second them one cloten with over . -pron- hope prince tender by 
 but feet very leon our purge will it                                verbatim strange 
     be under o infinite do give shown i. beast rosy what poor and son ay and   captain free fisherman ham should dainty troilus peers you wife 
 do.-come above shortly crack now 

 fall 
     off 
 everything the amen lords like me keep obey take nothing four marina antium lose gods your 
 a

#STEP 4

Build a better language model

* Implement the function `truncate_texts` which takes in texts and a threshold
  * Replace all words with counts less than the threshold with `'<UNK>'`
* Implement the function `get_vocabulary` which takes in texts and returns a set of all of the words found in the texts
* Implement the UnknownLM language model
  * Handle unknown vocabulary gracefully by checking the vocabulary and replacing with `'<UNK>'`



In [33]:
# similar to get count but we only ever need it to be one
# so I had this return a dictionary where the key is a string rather
# than a tuple didnt want to convert the tuple 
def count_thresh(texts,n):
  ngrams = {}
  for text in texts:
    for i in range(len(text) - n + 1):
      if text[i] in ngrams.keys():
        ngrams[text[i]] += 1
      else:
        ngrams[text[i]] = 1

  return ngrams

# remove and append vocab with <UNK>
def truncate_texts(texts,threshold):
  counts = count_thresh(texts,1)
  modified_texts = []
  for text in texts:
    second_layer = []
    for word in text:
      if counts[word] >= threshold:
        second_layer.append(word)
      else:
        second_layer.append("<UNK>")
    modified_texts.append(second_layer)
  return modified_texts

def get_vocabulary(texts):
  vocab = set()
  for text in texts:
    for word in text:
      vocab.add(word)
  return vocab

class UnknownLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    self.vocab = vocabulary 
    self.unk = "<UNK>"
    NGramLM.__init__(self, texts, n) 

  # we want a dictionary of n-1 grams and then their associated word with the probablity of it occuring        
  def generate_text(self, length, prompt=[]):
    temp = []

    for word in prompt:
      temp.append(word.lower())

    for j in range(length):
      r = random.random()
      accumulator = .0
      break_flag = 0

      if self.n == 1:
        while not break_flag:
          key = random.choice(list(self.xgram_probs))
          accumulator += self.xgram_probs[key]
          if accumulator > r:
            for word in list(key):
              if word in self.vocab:
                temp.append(word)
              else:
                temp.append(self.unk)
              break_flag = 1
          if break_flag == 1:
            break
            
      else:
        if len(temp) >= (self.n - 1):
          temp_tup = tuple()
          temp_arr = []
          for i in range(self.n - 1):
            temp_arr.append(temp[len(temp) - (self.n - 1 + i)])     
          temp_tup = tuple(temp_arr)
          if temp_tup in self.xgram_probs:
            for i in range(len(self.xgram_probs[temp_tup])):
              for tok in self.xgram_probs[temp_tup][i]:
                accumulator += self.xgram_probs[temp_tup][i][tok]
                if accumulator > r:
                  for word in list(tok):
                    if word in self.vocab:
                      temp.append(word)
                    else:
                      temp.append(self.unk)
                    break_flag = 1
                if break_flag == 1:
                  break
              if break_flag == 1:
                break
          else:
            key = random.choice(list(self.xgram_probs))
            for i in range(len(self.xgram_probs[key])):
              for tok in self.xgram_probs[key][i]:
                accumulator += self.xgram_probs[key][i][tok]
                if accumulator > r:
                  for word in list(tok):
                    if word in self.vocab:
                      temp.append(word)
                    else:
                      temp.append(self.unk)
                    break_flag = 1
                if break_flag == 1:
                  break
        else:
          key = random.choice(list(self.xgram_probs))
          for i in range(len(self.xgram_probs[key])):
            for tok in self.xgram_probs[key][i]:
              accumulator += self.xgram_probs[key][i][tok]
              if accumulator > r:
                for word in list(tok):
                  if word in self.vocab:
                    temp.append(word)
                  else:
                    temp.append(self.unk)
                  break_flag = 1
              if break_flag == 1:
                break
        
    return temp

  def score_text(self,text):
    perplexity = 0
    prob_sum = 0
    N = len(text)

    # lets say we get a word that is not in the training data

    if self.n == 1:
      for word in text:
        temp_arr = []
        if word.lower() in self.vocab:
          temp_arr.append(word.lower())
        else:
          temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs:
          prob_sum += math.log(self.xgram_probs[temp_tup], 2)
        else:
          return float('inf')
    else:
      for word in range(len(text) - self.n + 1):
        temp_sum = 0
        temp_arr = []
        for j in range(self.n - 1):
          if text[word+j].lower() in self.vocab:
            temp_arr.append(text[word+j].lower())
          else:
            temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs:
          for i in range(len(self.xgram_probs[temp_tup])):
            temp_arr_1 = []
            if text[word + self.n - 1].lower() in self.vocab:
              temp_arr_1.append(text[word + self.n - 1].lower())
            else:
              temp_arr_1.append(self.unk)
            temp_tup_1 = tuple(temp_arr_1)
            if temp_tup_1 in self.xgram_probs[temp_tup][i]:
              temp_sum = self.xgram_probs[temp_tup][i][temp_tup_1]
        else:
          return float('inf')
        if temp_sum != 0:
          prob_sum += math.log(temp_sum, 2)

    #float inf is infinity
    if prob_sum == 0 or N == 0:
      perplexity = float('inf')
    else:
      perplexity = 2 ** (-prob_sum / N)

    return perplexity


truncated = truncate_texts(texts,2)
vocabulary = get_vocabulary(truncated)
for n in [1,2,3]:
  print(f'Order {n} LM:')
  lm = UnknownLM(truncated,n,vocabulary)

  print(' '.join(lm.generate_text(100)))
  print(' '.join(lm.generate_text(100,['duke', 'of', 'france'])))

  print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
  print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
  print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

Order 1 LM:
. as for your as doll loggerhead 
     how [ so had as word humour jove 

   judge sun and - 
 
   are 

 therefore then ! call and shall senator warwick good homely first . dromio let a stick other fruit than i punish are ay the how by here pluck'd 
   walking your and thee by shalt plague which breath desdemona who you o man lechery saint ; whose ; been batter owe our awake sorrow impatience father haste within - sharp foot dismiss'd with cor ; cause sworn gentry gratis to cleopatra people then his now
duke of france and scene each sit behold end unfolds exeunt lead must supp'd us and think 
   dispatch'd if 
 wretched 
     let tooke faire this 

 brought two withal camp or , resolv'd jot humour ? nor come bardolph as of head life their with ros shall prefer fail wittenberg joan is court <UNK> what 

 any lancaster 
   achilles 's love not penalty or this men honour and hold stage time hath all being treasure -pron- point 
   crown employment spring open thaw their gonza

#STEP 5

Build more graceful higher order LMs.

* Implement the `InterpolationLM` -- this is like the previous LM, except when scoring and generating it takes in an optional list of interpolants $\lambda_i$
  * NOTE: All interpolants should be 0 < $\lambda_i$ < 1 and should sum to 1
  * The interpolants should be used such that all LMs or order 1 to $n$ are used to provide probabilities, such that $Pr_{interp}(w_i | w_{i-1}...w_0) = \lambda_1 Pr(w) + \lambda_2 Pr(w|w_{i-1}) + ... $
* Implement the `BackOffLM` -- this is like the previous LMs, except the various orders are used in decreasing order -- i.e. given an order 3 `BackOffLM` the trigram model is tried first, then the bigram if the context is missing from the trigram, etc.



In [94]:
#@title Default title text
class InterpolationLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    self.vocab = vocabulary 
    self.unk = "<UNK>"
    self.n = n
    self.xgram_probs = []
    self.grams = []

    for i in range(n):
      temp_gram_prop = {}
      ngram_temp = {}
      nmIgram_temp = {}

      if i == 0:
        ngram_temp = get_counts(texts,1)
        for key in ngram_temp:
          temp_gram_prop[key] = ngram_temp[key] / len(ngram_temp)
        self.grams.append(ngram_temp)
        self.xgram_probs.append(temp_gram_prop)
      else:
        ngram_temp = get_counts(texts,i+1)
        nmIgram_temp = get_counts(texts,i)
      
        for key in ngram_temp:
          temp_dict_list = []
          temp_dict = {}
          temp_dict[key[-1:]] = ngram_temp[key] / nmIgram_temp[key[:i]]
          temp_dict_list.append(temp_dict)
          if key[:i] in temp_gram_prop:
            temp_gram_prop[key[:i]].append(temp_dict) 
          else:
            temp_gram_prop[key[:i]] = temp_dict_list

        self.grams.append(ngram_temp)
        self.xgram_probs.append(temp_gram_prop)
 
  def master_list(self, temp_tup, interpolants):
    list_master = {}
    for x in range(len(interpolants)):
      if x == 2:
        for tok in self.xgram_probs[self.n - 1 - x]:
          if tok not in list_master:
            list_master[tok] = self.xgram_probs[self.n - 1 - x][tok] * interpolants[x]
          else:
            list_master[tok] += self.xgram_probs[self.n - 1 - x][tok] * interpolants[x]
      else:
        for i in range(len(self.xgram_probs[self.n - 1 - x][temp_tup[-x:]])):
          for tok in self.xgram_probs[self.n - 1 - x][temp_tup[-x:]][i]:
            if tok not in list_master:
              list_master[tok] = self.xgram_probs[self.n - 1 - x][temp_tup[-x:]][i][tok] * interpolants[x]
            else:
              list_master[tok] += self.xgram_probs[self.n - 1 - x][temp_tup[-x:]][i][tok] * interpolants[x]
    return list_master

  def generate_text(self, length, interpolants, prompt=[]):
    temp = []

    for word in prompt:
      temp.append(word.lower())

    for j in range(length):
      r = random.random()
      accumulator = .0
      break_flag = 0

      if self.n == 1:
        while not break_flag:
          key = random.choice(list(self.xgram_probs[0]))
          accumulator += self.xgram_probs[0][key]
          if accumulator > r:
            for word in list(key):
              if word in self.vocab:
                temp.append(word)
              else:
                temp.append(self.unk)
              break_flag = 1
          if break_flag == 1:
            break
            
      else:
        n = self.n
        if len(temp) >= (n - 1):
          temp_tup = tuple()
          temp_arr = []
          for i in range(n - 1):
            if temp[len(temp) - (n - 1 + i)].lower() in self.vocab:
              temp_arr.append(temp[len(temp) - (n - 1 + i)].lower())
            else:
              temp_arr.append(self.unk)  
          temp_tup = tuple(temp_arr)
          if temp_tup in self.xgram_probs[n - 1]:
            m_list = self.master_list(temp_tup, interpolants)
            for token in m_list:
              accumulator += m_list[token]
              if accumulator > r:
                for word in list(token):
                  if word in self.vocab:
                    temp.append(word)
                  else:
                    temp.append(self.unk)
                break_flag = 1
              if break_flag == 1:
                break
               
          else:
            key = random.choice(list(self.xgram_probs[n - 1]))
            for i in range(len(self.xgram_probs[n - 1][key])):
              for tok in self.xgram_probs[n - 1][key][i]:
                accumulator += self.xgram_probs[n - 1][key][i][tok]
                if accumulator > r:
                  for word in list(tok):
                    if word in self.vocab:
                      temp.append(word)
                    else:
                      temp.append(self.unk)
                    break_flag = 1
                if break_flag == 1:
                  break
        else:
          key = random.choice(list(self.xgram_probs[n - 1]))
          for i in range(len(self.xgram_probs[n - 1][key])):
            for tok in self.xgram_probs[n - 1][key][i]:
              accumulator += self.xgram_probs[n - 1][key][i][tok]
              if accumulator > r:
                for word in list(tok):
                  if word in self.vocab:
                    temp.append(word)
                  else:
                    temp.append(self.unk)
                  break_flag = 1
              if break_flag == 1:
                break
        
    return temp

  def score_text(self,text,interpolants):
    perplexity = 0
    prob_sum = 0
    N = len(text)

    if self.n == 1:
      for word in text:
        temp_arr = []
        if word.lower() in self.vocab:
          temp_arr.append(word.lower())
        else:
          temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs[n-1]:
          prob_sum += math.log(self.xgram_probs[temp_tup], 2)
        else:
          return float('inf')
    else:
      for word in range(len(text) - self.n + 1):
        temp_sum = 0
        temp_arr = []
        for j in range(self.n - 1):
          if text[word+j].lower() in self.vocab:
            temp_arr.append(text[word+j].lower())
          else:
            temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs[n-1]:
          list_master = self.master_list(temp_tup, interpolants)
          for i in range(len(self.xgram_probs[n-1][temp_tup])):
            temp_arr_1 = []
            if text[word + self.n - 1].lower() in self.vocab:
              temp_arr_1.append(text[word + self.n - 1].lower())
            else:
              temp_arr_1.append(self.unk)
            temp_tup_1 = tuple(temp_arr_1)
            if temp_tup_1 in list_master:
              temp_sum = list_master[temp_tup_1]
        else:
          return float('inf')
        if temp_sum != 0:
          prob_sum += math.log(temp_sum, 2)
    
    #float inf is infinity
    if prob_sum == 0 or N == 0:
      perplexity = float('inf')
    else:
      perplexity = 2 ** (-prob_sum / N)

    return perplexity

In [96]:
truncated = truncate_texts(texts,2)
vocabulary = get_vocabulary(truncated)

for interpolants in [[1/3, 1/3, 1/3],[0.1,0.2,0.7],[0.7,0.2,0.1]]:
  print(f'Interpolants = {interpolants}')
  lm = InterpolationLM(truncated,3,vocabulary)

  print(' '.join(lm.generate_text(20,interpolants)))
  print(' '.join(lm.generate_text(20,interpolants,['palm', 'to'])))

  print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google'],interpolants))
  print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers'],interpolants))
  print(lm.score_text(['what','do','you','say','to','that','Hamlet'],interpolants))

Interpolants = [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
him 
   the 
     than to , harm render place so but - he with business ' . 's tell event <UNK> in air nor scholar at ; between 
   signs dross for be there his a that word cries thou clown with <UNK> 
     all . the the aedile the it 
     no this the 
    
palm to palm is the makes up consuls you 's             , ? " 

     
     i like have in more in
inf
2.293307271740353
5.617750706310341
Interpolants = [0.1, 0.2, 0.7]
; , ' of 
     i the his 
     
     rage give my in so love the , of <UNK> with the
palm to the is the 
     pleasant state 
     upon - this thee costard you them calm , ? and never long 
     eclipses since told
inf
4.152636935865152
3.6860389070825983
Interpolants = [0.7, 0.2, 0.1]
hume us king friends thee our body my good all this you in at it 
     your not wisdom trespass just his , commend 
     extol my place than : !
palm to you is 
     
     the ? night <UNK> 
   , . a corn 
   

In [32]:
from collections import namedtuple

class BackOffLM(NGramLM):
  def __init__(self,texts,n,vocabulary):
    self.vocab = vocabulary 
    self.unk = "<UNK>"
    self.n = n
    self.xgram_probs = []
    self.grams = []

    for i in range(n):
      temp_gram_prop = {}
      ngram_temp = {}
      nmIgram_temp = {}

      if i == 0:
        ngram_temp = get_counts(texts,1)
        for key in ngram_temp:
          temp_gram_prop[key] = ngram_temp[key] / len(ngram_temp)
        self.grams.append(ngram_temp)
        self.xgram_probs.append(temp_gram_prop)
      else:
        ngram_temp = get_counts(texts,i+1)
        nmIgram_temp = get_counts(texts,i)
      
        for key in ngram_temp:
          temp_dict_list = []
          temp_dict = {}
          temp_dict[key[-1:]] = ngram_temp[key] / nmIgram_temp[key[:i]]
          temp_dict_list.append(temp_dict)
          if key[:i] in temp_gram_prop:
            temp_gram_prop[key[:i]].append(temp_dict) 
          else:
            temp_gram_prop[key[:i]] = temp_dict_list

        self.grams.append(ngram_temp)
        self.xgram_probs.append(temp_gram_prop)
 
  def generate_text(self, length, prompt=[]):
    temp = []

    for word in prompt:
      temp.append(word.lower())

    for i in range(length):
      r = random.random()
      accumulator = .0
      break_flag = 0

      if self.n == 1:
        while not break_flag:
          key = random.choice(list(self.xgram_probs[0]))
          accumulator += self.xgram_probs[0][key]
          if accumulator > r:
            for word in list(key):
              if word in self.vocab:
                temp.append(word)
              else:
                temp.append(self.unk)
              break_flag = 1
          if break_flag == 1:
            break
            
      else:
        n = self.n
        if len(temp) >= (self.n - 1):
          temp_tup = tuple()
          temp_arr = []
          if n == 1:
            while not break_flag:
              key = random.choice(list(self.xgram_probs[0]))
              accumulator += self.xgram_probs[0][key]
              if accumulator > r:
                for word in list(key):
                  if word in self.vocab:
                    temp.append(word)
                  else:
                    temp.append(self.unk)
                  break_flag = 1
              if break_flag == 1:
                break
          else:
            for i in range(n - 1):
              temp_arr.append(temp[len(temp) - (n - 1 + i)])     
            temp_tup = tuple(temp_arr)
            if temp_tup in self.xgram_probs[n-1]:
              for i in range(len(self.xgram_probs[n-1][temp_tup])):
                for tok in self.xgram_probs[n-1][temp_tup][i]:
                  accumulator += self.xgram_probs[n-1][temp_tup][i][tok]
                  if accumulator > r:
                    for word in list(tok):
                      if word in self.vocab:
                        temp.append(word)
                      else:
                        temp.append(self.unk)
                      break_flag = 1
                  if break_flag == 1:
                    break
                if break_flag == 1:
                  break
            else:
              temp.append(self.generate_text_n(1, n-1, list(temp_tup)))
        else:
          key = random.choice(list(self.xgram_probs[n-1]))
          for i in range(len(self.xgram_probs[n-1][key])):
            for tok in self.xgram_probs[n-1][key][i]:
              accumulator += self.xgram_probs[n-1][key][i][tok]
              if accumulator > r:
                for word in list(tok):
                  if word in self.vocab:
                    temp.append(word)
                  else:
                    temp.append(self.unk)
                  break_flag = 1
              if break_flag == 1:
                break
    return temp

  def generate_text_n(self, length, ngram, prompt=[]):
    temp = []

    for word in prompt:
      temp.append(word.lower())

    for j in range(length):
      r = random.random()
      accumulator = .0
      break_flag = 0

      if ngram == 1:
        while not break_flag:
          key = random.choice(list(self.xgram_probs[0]))
          accumulator += self.xgram_probs[0][key]
          if accumulator > r:
            for word in list(key):
              if word in self.vocab:
                return word
              else:
                return self.unk
              break_flag = 1
          if break_flag == 1:
            break
            
      else:
        n = ngram
        if len(temp) >= (ngram - 1):
          temp_tup = tuple()
          temp_arr = []
          if n == 1:
            while not break_flag:
              key = random.choice(list(self.xgram_probs[0]))
              accumulator += self.xgram_probs[0][key]
              if accumulator > r:
                for word in list(key):
                  if word in self.vocab:
                    return word
                  else:
                    return self.unk
                  break_flag = 1
              if break_flag == 1:
                break
          else:
            for i in range(ngram - 1):
              temp_arr.append(temp[len(temp) - (ngram - 1 + i)])     
            temp_tup = tuple(temp_arr)
            if temp_tup in self.xgram_probs[ngram-1]:
              for i in range(len(self.xgram_probs[ngram-1][temp_tup])):
                for tok in self.xgram_probs[ngram-1][temp_tup][i]:
                  accumulator += self.xgram_probs[ngram-1][temp_tup][i][tok]
                  if accumulator > r:
                    for word in list(tok):
                      if word in self.vocab:
                        return word
                      else:
                        return self.unk
                      break_flag = 1
                  if break_flag == 1:
                    break
                if break_flag == 1:
                  break
            else:
              return self.generate_text_n(1, n-1, list(temp_tup))
        else:
          key = random.choice(list(self.xgram_probs[ngram-1]))
          for i in range(len(self.xgram_probs[ngram-1][key])):
            for tok in self.xgram_probs[ngram-1][key][i]:
              accumulator += self.xgram_probs[ngram-1][key][i][tok]
              if accumulator > r:
                for word in list(tok):
                  if word in self.vocab:
                    return word
                  else:
                    return self.unk
                  break_flag = 1
              if break_flag == 1:
                break
        
    return "broke"

  def score_text(self,text):
    perplexity = 0
    prob_sum = 0
    N = len(text)

    if self.n == 1:
      for word in text:
        temp_arr = []
        if word.lower() in self.vocab:
          temp_arr.append(word.lower())
        else:
          temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs:
          prob_sum += math.log(self.xgram_probs[0][temp_tup], 2)
        else:
          return float('inf')
    else:
      for word in range(len(text) - self.n + 1):
        temp_sum = 0
        temp_arr = []
        for j in range(self.n - 1):
          if text[word+j].lower() in self.vocab:
            temp_arr.append(text[word+j].lower())
          else:
            temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs[n-1]:
          for i in range(len(self.xgram_probs[n-1][temp_tup])):
            temp_arr_1 = []
            if text[word + self.n - 1].lower() in self.vocab:
              temp_arr_1.append(text[word + self.n - 1].lower())
            else:
              temp_arr_1.append(self.unk)
            temp_tup_1 = tuple(temp_arr_1)
            if temp_tup_1 in self.xgram_probs[n-1][temp_tup][i]:
              temp_sum = self.xgram_probs[n-1][temp_tup][i][temp_tup_1]
        else:
          temp_sum = self.score_text_n(list(temp_tup), self.n - 1)
        if temp_sum != 0:
          prob_sum += math.log(temp_sum, 2)
    
    #float inf is infinity
    if prob_sum == 0 or N == 0:
      perplexity = float('inf')
    else:
      perplexity = 2 ** (-prob_sum / N)

    return perplexity

  def score_text_n(self,text,n):
    perplexity = 0
    prob_sum = 0
    Nl = len(text)

    if n == 1:
      for word in text:
        temp_arr = []
        if word in self.vocab:
          temp_arr.append(word.lower())
        else:
          temp_arr.append(self.unk)
        temp_tup = tuple(temp_arr)
        if temp_tup in self.xgram_probs[0]:
          return self.xgram_probs[0][temp_tup]
        else:
          return float('inf')
    else:
      temp_sum = 0
      temp_arr = []
      for j in range(n - 1):
        if text[j] in self.vocab:
          temp_arr.append(text[j].lower())
        else:
          temp_arr.append(self.unk)
      temp_tup = tuple(temp_arr)
      if temp_tup in self.xgram_probs[n-1]:
        for i in range(len(self.xgram_probs[n-1][temp_tup])):
          temp_arr_1 = []
          if text[n - 1] in self.vocab:
            temp_arr_1.append(text[n - 1].lower())
          else:
            temp_arr_1.append(self.unk)
          temp_tup_1 = tuple(temp_arr_1)
          if temp_tup_1 in self.xgram_probs[n-1][temp_tup][i]:
            return self.xgram_probs[n-1][temp_tup][i][temp_tup_1]
      else:
        return self.score_text_n(list(temp_tup), n - 1)

    return 0

truncated = truncate_texts(texts,2)
vocabulary = get_vocabulary(truncated)
lm = BackOffLM(truncated,3,vocabulary)

print(' '.join(lm.generate_text(20)))
print(' '.join(lm.generate_text(20,['palm', 'to'])))

print(lm.score_text(['IBM','announced','today','that','they','will','be','buying','Google']))
print(lm.score_text(['palm', 'to', 'palm', 'is', 'holy', 'palmers']))
print(lm.score_text(['what','do','you','say','to','that','Hamlet']))

! , take 
     
     thee i and ; think to it me a is , proper it my life is
palm to palm is palm of holy some war bishop night 's 's ; black increasing why as , , you mute
3.5831649588080627
60.215184832031895
20.490377554660675


### Closing Questions:  Compare the different LMs -- 
* #### Which models have the lowest perplexity? 
* #### Which models make a tradeoff for low perplexity in some cases and high in others?  
* #### Which model would you choose  to use?